In [3]:
import SimpleITK as sitk
import pandas as pd
import numpy as np
import tifffile

from pathlib import Path

#%matplotlib inline
#%matplotlib notebook
%matplotlib widget

import matplotlib.pyplot as plt
import gui
from math import ceil

In [4]:
def read_3d_volume(fileName, nx, ny, nz, dtype=np.float32):

    #print('Reading 3D volume file: ', fileName)
    f = open(fileName, "rb")
    data = np.fromfile(f, dtype=dtype)

    #size = nx*ny*nz
    shape = (nz, ny, nx)

    return data.reshape(shape)

In [5]:

input_path = Path('c:\\Users\\fe0968\\Documents\\data\\batteries\\segmentation')
data_path = input_path / 'volumes'
labels_path = input_path / 'labels'
masks_path = input_path / 'masks'

In [6]:
dataset = 'GFD_100'
frame = '01'

In [62]:
#plt.imshow(vol[200])
#plt.show()

In [9]:
vol = tifffile.imread(str(data_path / f'{dataset}_tomo{frame}.tif'))
labels = tifffile.imread(str(labels_path / f'{dataset}_tomo{frame}.tif'))
electr_mask = tifffile.imread(str(masks_path / f'{dataset}_electr.tif'))

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


In [10]:
plt.imshow(labels[200])
plt.show()

plt.imshow(electr_mask[200])
plt.show()

In [35]:
air = labels.copy()
air[air != 1] = 0

mem = labels.copy()
mem[mem != 2] = 0
mem_bin = mem / 2
#tifffile.imwrite(input_path / f'mem_orig.tif', mem)
mem_img = sitk.GetImageFromArray(mem_bin.astype(np.uint8))
# Dilate membrane to remove small holes
dilated_mem_img = sitk.BinaryDilate(mem_img, [3, 3, 3])
mem = sitk.GetArrayFromImage(dilated_mem_img)

#tifffile.imwrite(input_path / f'mem_close.tif', mem)

gask = labels.copy()
gask[gask != 3] = 0


In [36]:
electr = electr_mask / 255 - mem - gask/3
electr[electr<0] = 0

In [61]:
vol_img = sitk.GetImageFromArray(vol.astype(np.uint8))
electr_img = sitk.GetImageFromArray(electr.astype(np.uint8))

# Separate parts
stats = sitk.LabelShapeStatisticsImageFilter()
parts_img = sitk.ConnectedComponent(electr_img)
stats.Execute(parts_img)

# Look at the distribution of sizes of connected components
label_sizes = [ stats.GetNumberOfPixels(l) for l in stats.GetLabels() if l != 0]

In [50]:
gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(vol_img, parts_img)],                   
                      title_list = ['Cleaned Binary Segmentation'], figure_size=(8,4));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
print(stats.GetLabels())

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22)


In [43]:
print(label_sizes)

[99328182, 7646307, 1, 1, 1, 1, 1, 1, 45954, 9, 7, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2]


In [74]:
largest_component_label= np.argmax(np.array(label_sizes))
electr_volume = label_sizes[largest_component_label]
largest_component_binary_image = (parts_img == (largest_component_label+1))

In [53]:
gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(vol_img, largest_component_binary_image)],                   
                      title_list = ['Cleaned Binary Segmentation'], figure_size=(8,4));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
masked_bubbles_img = largest_component_binary_image * sitk.GetImageFromArray(air.astype(np.uint8))

m = 3
cleaned_bubbles_img = sitk.BinaryOpeningByReconstruction(masked_bubbles_img, [m, m, m])
cleaned_bubbles_img = sitk.BinaryClosingByReconstruction(cleaned_bubbles_img, [m, m, m])

gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(vol_img, masked_bubbles_img)],                   
                      title_list = ['Cleaned Binary Segmentation'], figure_size=(8,4));

C:\Users\fe0968\Documents\batteries\gui.py:464: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self.fig, self.axes = plt.subplots(row_num,col_num,figsize=figure_size)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
stats = sitk.LabelShapeStatisticsImageFilter()
stats.ComputeOrientedBoundingBoxOn()
bubbles_img = sitk.ConnectedComponent(cleaned_bubbles_img)
stats.Execute(bubbles_img)

# Look at the distribution of sizes of connected components (bacteria).
bubble_sizes = [ stats.GetNumberOfPixels(l) for l in stats.GetLabels() if l != 0]

plt.figure()
plt.hist(bubble_sizes,bins=200)
plt.title("Distribution of XObject Sizes")
plt.xlabel("size in pixels")
plt.ylabel("number of objects")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(vol_img, bubbles_img)],                   
                      title_list = ['Cleaned Binary Segmentation'], figure_size=(8,4));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
print(bubble_sizes)

[2996158, 776, 11247, 116681, 11019, 1785133, 32110, 18880, 13612, 2626, 3009, 23876, 4865, 13776, 130929, 17668, 46788, 10978, 9714, 473002, 41062, 736, 10025, 15105, 26389, 9057, 118233, 23919, 21759, 19543, 7982, 18426, 32690, 69481, 10988, 26646, 2319, 287310, 1784, 3025, 15849, 13012, 38555, 57946, 1741, 1662, 3026, 539, 13725, 28281, 1763, 40918, 4004, 71609, 57914, 1353, 24225, 81509, 11088, 392]


In [77]:
volume_fraction = np.sum(np.array(bubble_sizes)) / electr_volume
print('Volume fraction: 'volume_fraction*100)

6.9853659457896855


In [92]:
stats_list = [ (stats.GetPhysicalSize(i),
               stats.GetElongation(i),
               stats.GetFlatness(i),
               stats.GetRoundness(i),
               stats.GetOrientedBoundingBoxSize(i)[0],
               stats.GetOrientedBoundingBoxSize(i)[2]
               ) for i in stats.GetLabels()]
cols=["Volume (pixels)",
      "Elongation",
      "Flatness",
      "Roundness",
      "OBB Minimum Size (pixels)",
      "OBB Maximum Size (pixels)"
     ]

# Create the pandas data frame and display descriptive statistics.
res_stats = pd.DataFrame(data=stats_list, index=stats.GetLabels(), columns=cols)
res_stats.describe()

,Volume (pixels),Elongation,Flatness,Roundness,OBB Minimum Size (pixels),OBB Maximum Size (pixels)
count,6.000000e+01,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.156406e+05,1.739880,1.785802,0.635718,31.685916,78.223032
std,4.463186e+05,0.502162,0.973817,0.141901,24.232153,85.714769
min,3.920000e+02,1.070475,1.028950,0.206344,4.490952,16.009918
25%,4.649750e+03,1.414694,1.375593,0.541910,17.922601,42.498111
50%,1.547700e+04,1.638255,1.581531,0.623546,26.128452,59.064790
75%,3.914575e+04,1.900625,1.951043,0.728338,35.284451,80.629030
max,2.996158e+06,3.351956,8.226546,0.967778,134.749430,516.479666


In [91]:
res_stats

,Volume (pixels),Elongation,Flatness,Roundness,OBB Minimum Size (pixels),OBB Maximum Size (pixels)
1,2996158.0,1.327618,8.226546,0.206344,78.424256,516.479666
2,776.0,1.698487,1.747724,0.721043,8.180032,23.361995
3,11247.0,2.725304,1.254676,0.753046,20.861366,53.513027
4,116681.0,1.873781,1.117956,0.498906,69.119189,125.500708
5,11019.0,1.844881,1.382494,0.775972,21.370254,47.112624
6,1785133.0,2.458974,2.385841,0.476909,134.749430,467.470964
7,32110.0,2.382648,2.288968,0.543430,21.194132,105.763214
8,18880.0,1.731067,2.102348,0.689622,17.529435,66.723575
9,13612.0,1.495193,1.485169,0.722108,26.577376,46.859885
10,2626.0,3.351956,2.117268,0.496772,10.209956,65.339297
